<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/Colab'da_ger%C3%A7ek_zamanl%C4%B1_telefon_tespiti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Adım 1: Gerekli kütüphaneleri yükleme ve import
!pip install opencv-python numpy matplotlib ultralytics
import cv2
import numpy as np
import time
from IPython.display import display, Javascript, HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from ultralytics import YOLO
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Adım 2: Model yükleme
model = YOLO('yolov8n.pt')

# Telefon tespit geçmişi
phone_detection_history = {
    'last_detected': 0,
    'total_duration': 0,
    'start_time': None,
    'alert_level': 0
}

# Uyarı renkleri
ALERT_COLORS = {
    0: (0, 255, 0),   # Yeşil - Normal
    1: (0, 255, 255),  # Sarı - Düşük uyarı
    2: (0, 165, 255),  # Turuncu - Orta uyarı
    3: (0, 0, 255)     # Kırmızı - Yüksek uyarı
}

# Kamera erişim fonksiyonu
def get_video_feed(duration_sec=10):
    js = Javascript('''
    async function captureVideo(duration) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      const canvas = document.createElement('canvas');
      canvas.style.display = 'none';
      div.appendChild(canvas);
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;

      const start = Date.now();
      const frames = [];

      while (Date.now() - start < duration * 1000) {
        canvas.getContext('2d').drawImage(video, 0, 0);
        frames.push(canvas.toDataURL('image/jpeg', 0.7));
        await new Promise(r => setTimeout(r, 100));
      }

      stream.getVideoTracks()[0].stop();
      div.remove();
      return frames;
    }
    ''')
    display(js)
    return eval_js(f'captureVideo({duration_sec})')

# Düzeltilmiş sesli uyarı fonksiyonu
def play_audio_alert(duration):
    """Sesli uyarı çal (Colab uyumlu)"""
    try:
        if duration > 15:
            # Beep sesi oluştur
            import io
            import scipy.io.wavfile
            rate = 44100
            t = np.linspace(0, 0.5, int(rate*0.5))
            data = 0.5*np.sin(2*np.pi*880*t)
            data = np.concatenate([data, np.zeros_like(data)])
            with io.BytesIO() as f:
                scipy.io.wavfile.write(f, rate, data)
                display(Audio(f.getvalue(), autoplay=True))
        elif duration > 10:
            print("\a")  # Sistem bip sesi
    except Exception as e:
        print(f"Sesli uyarı hatası: {e}")

# Görsel uyarı fonksiyonu
def apply_visual_alerts(frame, duration):
    """Görsel uyarıları ekle"""
    alert_level = 0
    alert_text = "Telefon: YOK"

    if duration > 20:
        alert_level = 3
        alert_text = "⛔ ACİL BIRAKIN! (20+ saniye)"
    elif duration > 10:
        alert_level = 2
        alert_text = "⚠️ UZUN SÜRELİ KULLANIM (10+ saniye)"
    elif duration > 5:
        alert_level = 1
        alert_text = "🔔 Dikkat! Telefon kullanıyorsunuz"

    # Çerçeve rengi
    cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]),
                 ALERT_COLORS[alert_level], 10)

    # Uyarı metni
    cv2.putText(frame, alert_text, (10, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 0.9, ALERT_COLORS[alert_level], 2)

    return alert_level

# Ana tespit fonksiyonu
def detect_phone_and_measure():
    print("📱 Sistem başlatılıyor... Lütfen izin verin")

    frames = get_video_feed(15)  # 15 saniye kayıt

    if not frames:
        print("❌ Kamera erişimi sağlanamadı")
        return

    print("🔍 Analiz başladı...")

    for i, frame_data in enumerate(frames):
        binary_data = b64decode(frame_data.split(',')[1])
        img_array = np.frombuffer(binary_data, dtype=np.uint8)
        frame = cv2.imdecode(img_array, flags=cv2.IMREAD_COLOR)

        # Model çıktısını al
        results = model(frame, classes=[67])  # 67 = cell phone

        current_time = time.time()
        phone_detected = False

        for result in results:
            boxes = result.boxes
            if len(boxes) > 0:
                phone_detected = True
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(frame, f"Phone {box.conf[0]:.2f}",
                              (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX,
                              0.9, (0, 0, 255), 2)

                if phone_detection_history['start_time'] is None:
                    phone_detection_history['start_time'] = current_time
                phone_detection_history['last_detected'] = current_time

        if not phone_detected and phone_detection_history['start_time'] is not None:
            duration = current_time - phone_detection_history['start_time']
            phone_detection_history['total_duration'] += duration
            phone_detection_history['start_time'] = None

        current_duration = phone_detection_history['total_duration']
        if phone_detected and phone_detection_history['start_time'] is not None:
            current_duration += (current_time - phone_detection_history['start_time'])

        # Uyarı sistemlerini uygula
        alert_level = apply_visual_alerts(frame, current_duration)
        if i % 10 == 0:  # Her 10 framede bir sesli uyarı
            play_audio_alert(current_duration)

        info_text = f"Telefon: {'VAR' if phone_detected else 'YOK'}"
        info_text += f" | Süre: {current_duration:.1f}s"

        cv2.putText(frame, info_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        if i == len(frames)-1:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 8))
            plt.imshow(frame_rgb)
            plt.title(f"Son Durum - Toplam Süre: {current_duration:.1f}s")
            plt.axis('off')
            plt.show()

    print("\n📊 Analiz Sonuçları:")
    total_duration = phone_detection_history['total_duration']
    print(f"Toplam telefon kullanım süresi: {total_duration:.2f} saniye")

    if total_duration > 20:
        print("⛔ ACİL UYARI: 20 saniyeden fazla kullanım!")
    elif total_duration > 10:
        print("⚠️ UYARI: 10 saniyeden fazla kullanım")
    elif total_duration > 5:
        print("🔔 Bilgi: 5 saniyeden fazla kullanım")

# Sistemi çalıştır
detect_phone_and_measure()